In [10]:
import numpy as np
import mne

In [11]:
data=np.loadtxt("eeg-data.txt")
signal=data[:]

In [12]:
sfreq = 100
info = mne.create_info(ch_names=['EEG'], sfreq=sfreq, ch_types=['eeg'])
raw_data = mne.io.RawArray(data.reshape(1, -1), info=info)

Creating RawArray with float64 data, n_channels=1, n_times=3000
    Range : 0 ... 2999 =      0.000 ...    29.990 secs
Ready.


In [13]:
frequency_bands = {
    'Delta': (1, 4),
    'Theta': (4, 8),
    'Alpha': (8, 13),
    'Beta': (13, 30)
}
psds, freqs = mne.time_frequency.psd_array_welch(raw_data.get_data(), sfreq=sfreq, fmin=0.5, fmax=30)

Effective window size : 2.560 (s)


In [14]:

absolute_bandpowers = {}
relative_bandpowers = {}

total_power = np.sum(psds, axis=1)

for band_name, (fmin, fmax) in frequency_bands.items():
    logic_band = np.logical_and(freqs >= fmin, freqs <= fmax)
    band_power = np.sum(psds[:, logic_band], axis=1)
    absolute_bandpowers[band_name] = np.mean(band_power)
    relative_bandpowers[band_name] = np.mean(band_power / total_power)
    
print("Absolute Bandpowers:", absolute_bandpowers)
print("Relative Bandpowers:", relative_bandpowers)

max_relative_bandpower_band = max(relative_bandpowers, key=relative_bandpowers.get)
print(f"The highest relative bandpower: {max_relative_bandpower_band}")


Absolute Bandpowers: {'Delta': 384.2000368336033, 'Theta': 90.6767476656276, 'Alpha': 47.42722466517382, 'Beta': 12.54394591419076}
Relative Bandpowers: {'Delta': 0.53108273295183, 'Theta': 0.1253431815424361, 'Alpha': 0.06555902570725092, 'Beta': 0.017339595105228013}
The highest relative bandpower: Delta


In [15]:
psds, freqs = mne.time_frequency.psd_array_multitaper(data, sfreq=sfreq, fmin=0.5, fmax=30, adaptive=True, normalization='full', verbose=False)
bandpowers = {}

for band, (fmin, fmax) in frequency_bands.items():
    
    logic_band = np.logical_and(freqs >= fmin, freqs <= fmax)
    band_power = np.sum(psds[logic_band])
    bandpowers[band] = band_power

print("Bandpowers using Multitaper are :", bandpowers)

total_power = np.sum(psds)
relative_bandpowers = {}

for band, band_power in bandpowers.items():
    relative_power = band_power / total_power
    relative_bandpowers[band] = relative_power

print("Relative Bandpowers using Multitaper Method:", relative_bandpowers)

max_relative_band = max(relative_bandpowers, key=relative_bandpowers.get)
print(f"Frequency band with the highest relative bandpower using Multitaper Method: {max_relative_band}")

Bandpowers using Multitaper are : {'Delta': 5590.932088444477, 'Theta': 1093.3900779367464, 'Alpha': 495.4761151479902, 'Beta': 145.25095901157152}
Relative Bandpowers using Multitaper Method: {'Delta': 0.5009115301236374, 'Theta': 0.09796071357999252, 'Alpha': 0.044391470876826065, 'Beta': 0.013013551046486582}
Frequency band with the highest relative bandpower using Multitaper Method: Delta
